In [224]:
%%capture
%run 01_hash.ipynb
%run 02_transaction.ipynb
%run 05_block.ipynb

In [225]:
import subprocess as sub

## 99_contract

Our goal is to be able to run a small piece of code in a tx, just like Ethereum. This piece of code is also called a smart contract (sc). The sc is like another user, but in addition to the balance it also has storage for variables and the sc itself. 

The smart contract should be able to access the current tx and block information. These will get injected as `_tx` and `_b`.

The smart contract that we are going to run will contain the following logic:

In [226]:
cc = """
def init():
    storage r = 0
    s = 2
    h = 'hello how are you?'
    storage l = ['this', 'is', 'a', 'list']
    b = _b['number'] + 100

r += 5
storage time = _tx['time'] + 'XXXXX'
l.append('XXXXX')
"""

Temporary file name where code for execution will be stored.

In [227]:
F = 'temp.py'

This is injected into every execution to get all the variables used in the contract.

In [228]:
PRINT_VARS = """
\n
import json
_types = [str, float, int, list]                                                                         
out = {}                                                                                                 
keys = list(locals().keys())                                                                             
for k in keys:                                                                                           
    if not k.startswith('_'):                                                                            
        v = locals()[k]                                                                                  
        for t in _types:                                                                                 
            if isinstance(v, t): out[k] = [v, str(t)]                                                    
print(json.dumps(out)) 
"""
PRINT_VARS = PRINT_VARS.rstrip()

All `storage` vars used will be stored in the `STORAGE` dict.

In [229]:
STORAGE = {}

Inject transaction `tx` and `block` into the execution. These can be accessed in the sc.

In [230]:
def inject(tx, code, block=None): 
    tx = tx.json().replace('true', 'True').replace('false', 'False')
    b = block.json().replace('true', 'True').replace('false', 'False')
    return '_tx = '+tx+'\n'+'_b = '+b+'\n'+code

In [231]:
def save(code):
    with open(F, 'w+') as f: f.write(code)

In [232]:
def run():
    r = sub.check_output(f'python3 {F}', shell=True)
    return json.loads(r)

The deployment will run all the code in `init()`. The rest will run in `execution`.

In [233]:
def deployable(code):
    init = []
    lines = code.split('\n')
    for i,l in enumerate(lines):
        if 'init' in l: 
            for j in range(i+1, len(lines)):
                if lines[j][:1] == ' ': init.append(lines[j].strip())
                else                  : break
    init.append(PRINT_VARS)
    return '\n'.join(l for l in init)

Execution will run everything in the sc except what is inside `init()`.

In [234]:
def executable(code):
    exe = []
    for k,v in STORAGE.items():
        if 'str' in v[1]: exe.append(f'{k} = "{v[0]}"')
        else            : exe.append(f'{k} = {v[0]}')
    lines = code.split('\n')
    in_init = False
    for l in lines:
        if 'init' in l: in_init = True; continue
        if in_init:
            if l[:1] == ' ': continue
            else           : in_init = False
        else: exe.append(l)
    exe.append(PRINT_VARS)
    return '\n'.join(l for l in exe)

## Smart Contract

Stores all the necesary information for deploying and running smart contract `code`. 

The `tx` and `b` should be injected when the contract is run on the blockchain. This way the contract has always access to its tx and current block.

Stores all variables in `STORAGE`.

In [235]:
def parse(c): return code.replace('storage ', '')

In [236]:
class SmartContract(Hashable):
    def __init__(self, code):
        self.st_vars  = storage_vars(code)
        self.code     = parse(code)
        self.deployed = False
        self.time     = time.ctime()
        self.runs     = 0
        self.address  = sha(self.__dict__)
        
    def store(self, r):
        for k,v in r.items():
            if k in self.st_vars: STORAGE[k] = [v[0],v[1]]

    def exe(self, code, tx, b):
        ic = inject(tx, code, b)
        save(ic)
        r  = run()
        self.store(r)
        
    def deploy(self, tx, b): 
        dc = deployable(self.code)
        self.exe(dc, tx, b)
        self.deployed = True
        
    def run(self, tx, b): 
        assert self.deployed, 'deploy contract first'
        ic  = executable(self.code)
        self.exe(ic, tx, b)
        self.runs += 1

Notice that only the storage vars will be stored in `STORAGE`

Each smart contract has a unique address.

In [237]:
sc = SmartContract(code)
sc.address

'0xfe9aaf47a2dea123f920743e24f103c26e6e4cbcc2a3ff6be84ed42f55aa6bfc'

In [238]:
sc.deploy(tx1, b)
STORAGE

{'r': [0, "<class 'int'>"],
 'l': [['this', 'is', 'a', 'list'], "<class 'list'>"]}

In [239]:
sc.run(tx1, b)
STORAGE

{'r': [5, "<class 'int'>"],
 'l': [['this', 'is', 'a', 'list', 'XXXXX'], "<class 'list'>"],
 'time': ['Mon Apr 12 22:12:23 2021XXXXX', "<class 'str'>"]}